In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn

In [ ]:
#Run this cell to make plots look nice
import pylab
global_params = {
    'text.usetex' : False,
    'font.family' : 'serif',
    'font.sans-serif' : ['Helvetica'],
    'ps.usedistiller' : 'xpdf',
    'ps.distiller.res' : 3000,
    'axes.labelsize' : 13,
          #'text.fontsize' : 16,
    'legend.fontsize' : 10,
    'xtick.labelsize' : 12,
    'ytick.labelsize' : 12,
    'axes.linewidth': 2.0,
    'axes.grid':True,  
    'figure.figsize' : [7,5],
    'grid.linestyle':'--',
    'grid.color':'k',
    'grid.alpha':0.1}
pylab.rcParams.update(global_params) 


In this notebook you will explore how different unsupervised learning algorithms operate on the same data set

## Inspect the data

In [ ]:
#read in data
data=pd.read_csv('Unsupervised.csv')
data.head(5)

This data comes from a subset of the Bright Transient Survey (BTS) data set [cite this better]. There are only 2 classes, 'SN Ia' and 'SN II' which can be found under the 'type' column. For the guided part of the notebook we will be looking at the peak absolute magnitude ('peakabs') and fade ('fade') features.

In [ ]:
#To do: Create a pandas dataframe that only contains the columns 'ZTFID' 'peakabs' 'fade' and 'type'
relavant_data=...

Now that the data has been pared down we can explore its structure.

In [ ]:
#To do: Make a scatter plot showing each type separately
fig,ax=plt.subplots(figsize=(8,6))
SN1a_data=...
SN2_data=...
ax.scatter(....)
ax.legend()
plt.tight_layout()

By eye we can see that the two classes live in different areas of this plane. SNII tend to have longer fade time and are dimmer (larger peak magnitude). Also SN II have a larger spread compared to SN Ia. If you do not see these trends ask for help from the instructor. 

## Unsupervised learning

Now let's take a dive into the unsupervised learning algorithms! The first thing we should do is scale the data. Let's use the standard scaler provided by sklearn

In [ ]:
from sklearn.preprocessing import StandardScaler
# T0 do: apply the standard scaler to the data
# see docs at : https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
feature_data=... #get the data for only the features and cast them to floats
scaler=StandardScaler().fit(feature_data)
scaled_data=scaler.transform(feature_data)

## Kmeans

In [ ]:
from sklearn.cluster import KMeans
#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.k_means.html

The Kmeans algorithms main knob to turn is the number of cluster. Lets see what it looks like to use different number of clusters

In [ ]:
#To do: train kemans for 2,5 and 10 clusters

#example
ex_model=KMeans(n_clusters=15) #define a model
ex_model.fit(scaled_data)      #fit the model
ex_model.predict(scaled_data)  #get the classes

Now plot them and compare to the true classes

In [ ]:
#To do: Plot the true classes and 3 kmeans
#hint: use Kmeans.fit(scaled_data).predict(scaled_data) for your color map
fig,axs=plt.subplots(2,2,figsize=(8,6))
axs=axs.flatten()
axs[0].scatter(SN1a_data['peakabs'],SN1a_data['fade'],label='SN Ia',s=5)
axs[0].scatter(SN2_data['peakabs'],SN2_data['fade'],label='SN II',s=5)
axs[0].set_title('True Classes')
axs[0].legend()
n_clus=[...]
for i,model in enumerate(n_clus):
    axs[i+1].scatter(feature_data['peakabs'],feature_data['fade'],c=...)
    axs[i+1].set_title(f'N_cluster={n_clus[i]}')
plt.tight_layout()

How does it look? Can we determine the number of clusters systematically? KMeans is not well equipped to answer this question, keep this in mind for Gaussian Mixture

## DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
#https://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

DBSCAN does not take a number of clusters as a parameter. Instead it has epsilon and min samples as tunable parameters.

In [ ]:
#To do: Test a few different epsilon values and find out how many classes are created
fig,axs=plt.subplots(2,2,figsize=(8,6))
axs=axs.flatten()
axs[0].scatter(SN1a_data['peakabs'],SN1a_data['fade'],label='SN Ia',s=5)
axs[0].scatter(SN2_data['peakabs'],SN2_data['fade'],label='SN II',s=5)
axs[0].set_title('True Classes')
axs[0].legend()

epsilons=[...] #pick 3 values here

for i,eps in enumerate(epsilons):
    model=DBSCAN(...) #keep min_samples as the default
    model.fit(...)
    axs[i].scatter(feature_data['peakabs'],feature_data['fade'],c=model.predict(scaled_data),s=5)
    axs[i].set_title(f'epsilon= {eps}. Number of classes= {...}')


In [ ]:
#To do: Now do the same but vary min samples. Keep eps=0.2

Changing epsilon and minimum samples can dramaticly change the outcome. Now lets do some bad science and try to pick epsilon and minimum samples to match our data! (this is for academic purposes only).

Since the density of the two classes differ DBSCAN is not ideal for this data set.

## Gaussian Mixture Model

In [ ]:
from sklearn.mixture import GaussianMixture
# https://scikit-learn.org/stable/modules/generated/sklearn.mixture.GaussianMixture.html#sklearn.mixture.GaussianMixture

One advantage that Gaussian Mixture Model (GMM) has is its statistical basis. This implementaion of GMM has built in functions for the Bayesian and Akaike information criterion (BIC and AIC). These give you an idea of how good the model is. We can use this to determine how many clusters we should choose!

In [ ]:
# To do: Compute and Plot the BIC and AIC for a variety of cluster numbers
number_of_clusters=[1,2,3,4,5,6,7,8,9,10]
bic=[]
aic=[]
#example
example_model=GaussianMixture(n_components=1).fit(scaled_data)
ex_bic=example_model.bic(scaled_data)
ex_aic=example_model.aic(scaled_data)

fig,ax=plt.subplots(figsize=(6,4))
ax.plot(number_of_clusters,bic,label='BIC',marker='o')
ax.plot(number_of_clusters,aic,label='AIC',marker='o')
ax.legend()
plt.tight_layout()

In the plot we can see that the BIC is minimum at 3 and the AIC starts to get diminishing returns at 3. So this tells us that the number of cluster is probably 2,3 or 4.(If you don't see this ask for help) Now let's fit for those parameters and see what it looks like.

In [ ]:
#To do: Plot gmm models for 2,3 and 4 clusters.
fig,axs=plt.subplots(2,2,figsize=(8,6))
axs=axs.flatten()
axs[0].scatter(SN1a_data['peakabs'],SN1a_data['fade'],label='SN Ia',s=5)
axs[0].scatter(SN2_data['peakabs'],SN2_data['fade'],label='SN II',s=5)
axs[0].set_title('True Classes')
axs[0].legend()

number_of_clusters=[...] 

for i,n_clus in enumerate(number_of_clusters):
    model=GaussianMixture(n_components=...)
    model.fit(...)
    axs[i+1].scatter(feature_data['peakabs'],feature_data['fade'],c=model.predict(scaled_data),s=5)
    axs[i+1].set_title(f' Number of clusters= {n_clus}')
plt.tight_layout()

2 clusters doesn't look too bad! Now let's try to extract the gaussians from the model

In [ ]:
#function to plot a contour
def plot_contour(ax,center,cov,lvls=3,cmap='viridis'):
    '''
    ax:     matplotlib axis that you want the contour to be plotted on
    center: array of mean of gaussian [x_mean,ymean]
    cov:    covariance matrix
    sig:    int, number of contours to plot
    cmap:   Matplotlib color map
    '''
    x = np.arange(-21, -14, .1)
    y = np.arange(0, 120, .1)
    X, Y = np.meshgrid(x, y)
    xhat=center[0]
    yhat=center[1]
    sigx=cov[0][0]
    sigy=cov[1][1]
    A=(X-xhat)**2/((2*sigx)**2) #Ignoring covariance
    B=(Y-yhat)**2/(2*sigy**2)
    Z=np.exp(-(A+B))
    ax.contour(X, Y, Z,levels=lvls-1,linestyles='dashed',cmap=cmap)

In [ ]:
#To do: Plot the contours of the gaussians found by the model over the data
fig,ax=plt.subplots(figsize=(8,6))
model=GaussianMixture(n_components=...).fit(feature_data) #Use feature data instead of scaled data 
mean=...
cov=...
ax.scatter(SN1a_data['peakabs'],SN1a_data['fade'],label='SN Ia',s=5)
ax.scatter(SN2_data['peakabs'],SN2_data['fade'],label='SN II',s=5)
ax.set_title('True Classes')
plot_contour(...)
plot_contour(...)
ax.legend()

## Wrap up

Here is a plot showing all 3 algorithms at once. I hope you all have learned when each one can be used as well as the advantages and disadvantages of each!

In [ ]:
fig,axs=plt.subplots(2,2,figsize=(8,6))
axs=axs.flatten()
axs[0].scatter(SN1a_data['peakabs'],SN1a_data['fade'],label='SN Ia',s=5)
axs[0].scatter(SN2_data['peakabs'],SN2_data['fade'],label='SN II',s=5)
axs[0].set_title('True Classes')
axs[0].legend()


kmeans_model=KMeans(n_clusters=2).fit(scaled_data)
axs[1].scatter(feature_data['peakabs'],feature_data['fade'],c=kmeans_model.predict(scaled_data),s=5)
axs[1].set_title('Kmeans')



gmm_model=GaussianMixture(n_components=2).fit(scaled_data)
axs[2].scatter(feature_data['peakabs'],feature_data['fade'],c=gmm_model.predict(scaled_data),s=5)
axs[2].set_title('GMM')

db=DBSCAN(eps=0.15,min_samples=7).fit(scaled_data)
axs[3].scatter(feature_data['peakabs'],feature_data['fade'],c=db.labels_,s=5,cmap='turbo')
axs[3].set_title('DBSCAN')
plt.tight_layout()

## Bonus
Try your hand at these 2 things! 
1) Compute an accuracy for the various methods. Typically, you wouldn't know the classes in an unsupervised problem but we do here. See how they performed! 
2) Use different "important" features and see what they look like.